# About QR
**QR stands for (quick response)**. The smallest QR codes are 21x21 pixels, and the largest are 177x177. The sizes are called versions. **The 21x21 pixel size is version 1, 25x25 is version 2, and so on. The 177x177 size is version 40.**

In addition, **QR codes include error correction: when you encode the QR code, you also create some redundant data that will help a QR reader accurately read the code even if part of it is unreadable.** There are four levels of error correction that you can choose from:
1. The lowest is L, which allows the code to be read even if 7% of it is unreadable. 
* After that is M, which provides 15% error correction, 
* then Q, which provides 25%,
* finally H, which provides 30%.

The capacity of a given QR code depends on the version and error correction level, as well as on the type of data that you are encoding. **There are four data modes that a QR code can encode: numeric, alphanumeric, binary, or Kanji.**

# The QR Code Modes
This tutorial considers only 2 modes i.e, **Numeric mode and Alphanumeric mode.**
1. **Numeric mode** is for decimal digits 0 through 9.
2. **Alphanumeric mode** is for the decimal digits 0 through 9, as well as uppercase letters (not lowercase!), and the symbols `$, %, *, +, -, ., /, and :` as well as a space.

# Data Encoding

## Step 1: Choose the Error Correction Level
QR codes use Reed-Solomon error correction. Error correction codewords can be used to correct those errors. There are four levels of error correction: L, M, Q, H.  

| Error Correction Level | Error Correction Capability |
| :---: | :---: |
| L | Recovers 7% of data |
| M | Recovers 15% of data |
| Q | Recovers 25% of data |
| H | Recovers 30% of data |

In [52]:
import re
text=input("Enter the text to be encoded\n")
text=text.upper()
alpha_lst=re.findall('\D',text)
if len(alpha_lst)>0:
    print('Mode of encoding: ALPHANUMERIC')
    mode=1
else:
    print('Mode of encoding: NUMERIC')
    mode=0
ECL=eval(input("""Enter error correction level:
0 for L (Low)
1 for M (Medium)
2 for Q
3 for H (High)\n"""))
print("MODE SELECTED: {} and ERROR CORECTION LEVEL SELECTED: {}".format(mode,ECL))

Enter the text to be encoded
deepanjan saha
Mode of encoding: ALPHANUMERIC
Enter error correction level:
0 for L (Low)
1 for M (Medium)
2 for Q
3 for H (High)
0
MODE SELECTED: 1 and ERROR CORECTION LEVEL SELECTED: 0


## Step 2: Determine the Smallest Version for the Data
Each version has a maximum capacity, depending on the mode in use. In addition, the error correction
level restricts the capacity further.  
[Link to Character Capacities by Version, Mode, and Error Correction](#Character-Capacities-by-Version,-Mode,-and-Error-Correction)

# Character Capacities by Version, Mode, and Error Correction

In [53]:
# Displaying the table for character capacity
import pandas as pd
char_capacity=pd.read_csv('character_capacity.csv')
char_capacity.head(8)

version->   1   2    3    4    5    6    7    8    9  ...     31  \
0       Numericmode(L)  41  77  127  187  255  322  370  461  552  ...   4417   
1       Numericmode(M)  34  63  101  149  202  255  293  365  432  ...   3486   
2       Numericmode(Q)  27  48   77  111  144  178  207  259  312  ...   2473   
3       Numericmode(H)  17  34   58   82  106  139  154  202  235  ...   1897   
4  AlphanumericMode(L)  25  47   77  114  154  195  224  279  335  ...   2677   
5  AlphanumericMode(H)  20  38   61   90  122  154  178  221  262  ...   2113   
6  AlphanumericMode(Q)  16  29   47   67   87  108  125  157  189  ...   1499   
7  AlphanumericMode(H)  10  20   35   50   64   84   93  122  143  ...   1150   

     32    33    34    35    36    37    38    39    40  
0  4686  4965  5253  5529  5836  6153  6479  6743  7089  
1  3693  3909  4134  4343  4588  4775  5039  5313  5596  
2  2670  2805  2949  3081  3244  3417  3599  3791  3993  
3  2022  2157  2301  2361  2524  2625  2735  2927  3057  
4  2840  3009  3183  3351  3537  3729  3927  4087  4296  
5  2238  2369  2506  2632  2780  2894  3054  3220  3391  
6  1618  1700  1787  1867  1966  2071  2181  2298  2420  
7  1226  1307  1394  1431  1530  1591  1658  1774  1852  

[8 rows x 41 columns]

In [54]:
import numpy as np
max_capacity=np.zeros((40,4,4),dtype=np.uint16)
for i in range(40):
    for j in range(4):
        max_capacity[i,:,j]=char_capacity.iloc[4*j:4*j+4,i+1]

for i in range(40):
    if max_capacity[i,ECL,mode]>len(text):
        version=i
        break
print('Selected version is: {}'.format(version))

Selected version is: 0


## Step 3: Add the Mode Indicator
Each encoding mode has a four-bit mode indicator that identifies it.

| Mode Name | Mode Indicator |
| :---: | :---: |
|Numeric Mode | 0001 |
|Alphanumeric Mode | 0010 |
|Byte Mode | 0100 |
|Kanji Mode | 1000 |
|ECI Mode | 0111 |

In [55]:
mode_indicator_array=('0001','0010','0100','1000','0111')
mode_indicator=mode_indicator_array[mode]
print('mode indicator selected: {}'.format(mode_indicator))

mode indicator selected: 0010


## Step 4: Add the Character Count Indicator
The character count indicator is a string of bits that **represents the number of characters that are being
encoded.** The character count indicator must be **placed after the mode indicator.** Character count indicator must be a **certain number of bits long, depending on the QR version.**

Count the number of characters in the original input text, then convert that number into binary. The length of the character count indicator depends on the encoding mode and the QR code version that will be in use. **To make the binary string the appropriate length, pad it on the left with 0s.**

| | Numeric mode | Alphanumeric mode | Byte mode | Japanese mode |
| :-: | :-: | :-: | :-: | :-: |
| Versions 1 through 9 | 10 bits | 9 bits | 8 bits | 8 bits |
| Versions 10 through 26 | 12 bits | 11 bits | 16 bits | 10 bits |
| Versions 27 through 40 | 14 bits | 13 bits | 16 bits | 12 bits |

In [56]:
character_count_array=np.array([[10,9,8,8],
                               [12,11,16,10],
                               [14,13,16,12]],dtype=np.uint8)
if 1<=version+1<=9:
    character_count_len=character_count_array[0,mode]
elif 10<=version+1<=26:
    character_count_len=character_count_array[1,mode]
elif 27<=version+1<=40:
    character_count_len=character_count_array[2,mode]
character_count=bin(len(text))[2:].zfill(character_count_len)
print('Character count for text: {}'.format(character_count))

Character count for text: 000001110


## Step 5: Encode Using the Selected Mode
**Each mode has separate encoding.**  
[Link to numeric encoding](#Numeric-encoding)  
work needed to be done in numeric encoding
[Link to alphanumeric encoding](#Alphanumeric-encoding)

### Alphanumeric-encoding
1. Take a sample string say "HELLO WORLD" and split it into pairs of 2 i.e, 'HE','LL','O ','WO','RL' and 'D'.
* Find the index of each character in pair and convert it into a number i.e, H->17 and E->14, so equivalent number is `17*45+14=779`.
* convert that number into an 11-bit binary string i.e, 779->01100001011
* If you are encoding an odd number of characters, as we are here, take the numeric representation of the final character and convert it into a 6-bit binary string.

In [57]:
alphanumeric_lst=[i for i in '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ $%*+-./:']
l=len(text)
data=''
for i in range(0,l,2):
    if len(text[i:i+2])==2:
        data+=bin(alphanumeric_lst.index(text[i])*45+alphanumeric_lst.index(text[i+1]))[2:].zfill(11)
    else:
        data+=bin(alphanumeric_lst.index(text[i]))[2:].zfill(6)
print('Alphanumeric encoded data string is : {}'.format(data))

Alphanumeric encoded data string is : 01001010111010100011110011101100101101100001100001011111001111011001100000111


## Step 6: Break Up into 8-bit Codewords and Add Pad Bytes if necessary
After obtaining a string of bits that consists of the mode indicator, the character count indicator, and the data bits, it may be necessary to add 0s and pad bytes, **because the QR code specification requires that the bit string must completely fill the total capacity of the QR code.** So, follow the below steps:
* Determine the Required Number of Bits for this QR Code (form the below table)
* Add a Terminator of 0s if Necessary at end to make no. of bits multiple of 8.
* Add Pad Bytes if the String is Still too Short (11101100 00010001) these bytes are equivalent to 236 and 17, respectively.

In [58]:
# Displaying the data
import pandas as pd
ECT=pd.read_csv('error_correction_table.csv')
ECT.head(12)

Version and EC Level  Total Number of Data Codewords  \
0                   1-L                              19   
1                   1-M                              16   
2                   1-Q                              13   
3                   1-H                               9   
4                   2-L                              34   
5                   2-M                              28   
6                   2-Q                              22   
7                   2-H                              16   
8                   3-L                              55   
9                   3-M                              44   
10                  3-Q                              34   
11                  3-H                              26   

    EC Codewords Per Block  Number of Blocks in Group 1  \
0                        7                            1   
1                       10                            1   
2                       13                            1   
3                       17                            1   
4                       10                            1   
5                       16                            1   
6                       22                            1   
7                       28                            1   
8                       15                            1   
9                       26                            1   
10                      18                            2   
11                      22                            2   

    Number of Data Codewords in Each of Group 1's Blocks  \
0                                                  19      
1                                                  16      
2                                                  13      
3                                                   9      
4                                                  34      
5                                                  28      
6                                                  22      
7                                                  16      
8                                                  55      
9                                                  44      
10                                                 17      
11                                                 13      

    Number of Blocks in Group 2  \
0                             0   
1                             0   
2                             0   
3                             0   
4                             0   
5                             0   
6                             0   
7                             0   
8                             0   
9                             0   
10                            0   
11                            0   

    Number of Data Codewords in Each of Group 2's Blocks Total Data Codewords  
0                                                   0               (19*1)=19  
1                                                   0               (16*1)=16  
2                                                   0               (13*1)=13  
3                                                   0                 (9*1)=9  
4                                                   0               (34*1)=34  
5                                                   0               (28*1)=28  
6                                                   0               (22*1)=22  
7                                                   0               (16*1)=16  
8                                                   0               (55*1)=55  
9                                                   0               (44*1)=44  
10                                                  0               (17*2)=34  
11                                                  0               (13*2)=26

In [59]:
codeWord,ErrorBlock,nb1,wnb1,nb2,wnb2=ECT.iloc[4*version+ECL,1:7]
print('Total Codeword for version: {} and Error correction level: {} is {}'.format(version+1,ECL,codeWord))
combined_data=mode_indicator+character_count+data
l=len(combined_data)
print(l)
if l%8!=0:
    combined_data=combined_data+''.zfill(8-l%8)
    l=len(combined_data)
    padding=['11101100','00010001']
    for i in range(codeWord):
        if len(combined_data)==8*codeWord:
            break
        combined_data=combined_data+padding[i%2]
print('Encoded data: {}\nLength of encoded data: {} bits\nNo. of bits required for this version: {} bits'.format(
    combined_data,len(combined_data),codeWord*8))

Total Codeword for version: 1 and Error correction level: 0 is 19
90
Encoded data: 00100000011100100101011101010001111001110110010110110000110000101111100111101100110000011100000011101100000100011110110000010001111011000001000111101100
Length of encoded data: 152 bits
No. of bits required for this version: 152 bits


# Error Correction Coding

## Step 1: Break Data Codewords into Blocks if Necessary
Divide the codeword in small blocks of 8 bits.

In [60]:
codeblocks=[int(combined_data[i:i+8],base=2) for i in range(0,len(combined_data),8)]
print('8 bit blocks are: {}'.format(codeblocks))

8 bit blocks are: [32, 114, 87, 81, 231, 101, 176, 194, 249, 236, 193, 192, 236, 17, 236, 17, 236, 17, 236]


The error correction table mentions "group 1" and "group 2", as well as "number of blocks." This means that the data codewords must be split up into up to two groups, and within each group, the data codewords may be further broken up into blocks. The data codewords are broken up sequentially.

In [87]:
block_groups,count=[],0
for i in range(nb1):
    block_groups.append(codeblocks[count:count+wnb1])
    count+=wnb1
for i in range(nb2):
    block_groups.append(codeblocks[count:count+wnb2])
    count+=wnb2
print('Groups divided into blocks:\n')
for i in block_groups:
    print(i)

Groups divided into blocks:

[32, 114, 87, 81, 231, 101, 176, 194, 249, 236, 193, 192, 236, 17, 236, 17, 236, 17, 236]


## Understand Galois Field Arithmetic
1. Addition:  
GF(256) contains the numbers 0 through 255 inclusive. Mathematical operations in GF(256) are cyclical in nature, meaning that if a mathematical operation is performed within GF(256) that results in a number larger than 255, it will be necessary to use the modulo operation to get a number that is still in the Galois Field. Addition and subtraction in the Galois Field are performed by doing the addition or subtraction as normal, but then performing the modulo operation and **this is the same as performing the XOR operation.**
* Generate Powers of 2 using Byte-Wise Modulo 100011101(285):  
All numbers in GF(256) can be represented as 2 , where 0 <= n <= 255. The QR code specification says to use byte-wise modulo 100011101 arithmetic (where 100011101 is a binary number that is equivalent to 285 in decimal). This means that **when a number is 256 or larger, it should be XORed with 285.** Because all of the values can be represented with 2 as explained above, it is possible to logs and antilogs to simplify multiplication in GF(256).
* Understanding The Generator Polynomial:  
The message polynomial will be divided by a generator polynomial. The generator polynomial is a polynomial that is created by multiplying $(x-\alpha)\dots(x-\alpha^{n-1})$ where n is the number of error correction codewords that must be generated. As mentioned in the previous section, $\alpha$ is equal to 2. When the Exponents are Greater than or Equal to 256, in this case, **apply modulo 255 BEFORE combining like terms.** For example:
$$
\alpha^{258}x = \alpha^{\text{258%255}}x = \alpha^{3}x
$$
* Generating Error Correction Codewords:  
For each block group, the coefficients represent a message polynomial. From the version and error correction level (ECL) value, **find the number of error codeblocks needed.** To make sure that the exponent of the lead term of message polynomial doesn't become too small during the division, **multiply the message polynomial by $x^n$ where $n$ is the number of error correction codewords that are needed and divide it with generator polynomial, to find the remainder polynomial whose coefficients becomes error codewords.**

In [91]:
def log_function():
    a=[1]
    for i in range(1,256):
        a.append(a[i-1]*2)
        if a[i]>255:
            a[i]^=285
    return a
log=log_function()
def anti_log_function(log):
    a=[None]
    for i in range(1,256):
        a.append(log.index(i))
    return a
anti_log=anti_log_function(log)
def mul(x,y,log=log,anti_log=anti_log):
    if x==0 or y==0:
        return 0
    return log[(anti_log[x]+anti_log[y])%255]
def add(x,y):
    return x^y
def generator_function(eb,log=log,anti_log=anti_log):
    gen_poly=[1,1]
    for i in range(1,eb):
        temp=mul(gen_poly[-1],log[i])
        for j in range(i,0,-1):
            gen_poly[j]=add(mul(gen_poly[j],1),mul(gen_poly[j-1],log[i]))
        gen_poly.append(temp)
    return gen_poly
def poly_divide(msg,gen):
    gen_len=len(gen)
    msg_len=len(msg)
    msg.extend([0]*(gen_len-1))
    for k in range(msg_len):
        multiple=[mul(gen[i],msg[0]) for i in range(gen_len)]
        for i in range(gen_len):
            msg[i]=msg[i]^multiple[i]
        msg.remove(0)
    return msg
# poly_divide()
genrator_poly=generator_function(ErrorBlock)
error_block_groups=[]
for i in block_groups:
    bg=i.copy()
    error_block_groups.append(poly_divide(bg,genrator_poly))
print('Error codeblocks generated are:')
for i,j in enumerate(error_block_groups):
    print('Error codeblock no. {}:{}'.format(i,j))

Error codeblocks generated are:
Error codeblock no. 0:[78, 196, 90, 72, 13, 201, 253]


In [93]:
# Displaying all results
print(version,mode,ECL,'\n')
print(mode_indicator,character_count,data,'\n')
print(combined_data,'\n')
print(codeblocks,'\n')
for i,j in enumerate(block_groups):
    print('Codeblock no. {}:{}'.format(i,j))
for i,j in enumerate(error_block_groups):
    print('Error codeblock no. {}:{}'.format(i,j))
print('\n',codeWord,ErrorBlock,nb1,wnb1,nb2,wnb2,'\n')

0 1 0 

0010 000001110 01001010111010100011110011101100101101100001100001011111001111011001100000111 

00100000011100100101011101010001111001110110010110110000110000101111100111101100110000011100000011101100000100011110110000010001111011000001000111101100 

[32, 114, 87, 81, 231, 101, 176, 194, 249, 236, 193, 192, 236, 17, 236, 17, 236, 17, 236] 

Codeblock no. 0:[32, 114, 87, 81, 231, 101, 176, 194, 249, 236, 193, 192, 236, 17, 236, 17, 236, 17, 236]
Error codeblock no. 0:[78, 196, 90, 72, 13, 201, 253]

 19 7 1 19 0 0 

